<a href="https://colab.research.google.com/github/ClouDevops101/size-limit/blob/main/recommemdation-engine-udemy-v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# build a online courses recommendation system

In [45]:
# importing libraries
# src https://www.youtube.com/watch?v=ueKXSupHz6Q

In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
# Download stop words
import nltk
nltk.download('stopwords')
# Saving model to file
from numpy import save
from numpy import asarray
from numpy import load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
# show available language
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [48]:
# Load the data 
#from google.colab import files
#uploaded = files.upload()

In [49]:
#Store the data 
df = pd.read_csv('udemy-data-v2.csv')
# show the first 3 rows of data 
df['title'].head(3)

0                        NoSQL Databases For Beginners
1                            Tai Chi Foundation Course
2    Music Pedagogy: 7 advices to teach a musical i...
Name: title, dtype: object

In [50]:
df.shape


(3945, 13)

In [51]:
# Creat a list of important columns for the recommendation engine
columns= ['title','description','headline','category','subcategory','instructors_name','instructors_job_title']

In [52]:
# show the data 
df[columns].head(3)


,title,description,headline,category,subcategory,instructors_name,instructors_job_title
0,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple
1,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist
2,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De..."


In [53]:
# check for missing values in the important columns
df[columns].isnull().values.any()

# data cleaning is needed

True

In [54]:
# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(str(data['title'][i])+' '+str(data['description'][i])+' '+str(data['headline'][i])+' '+str(data['category'][i])+' '+str(data['subcategory'][i])+' '+str(data['instructors_name'][i])+' '+str(data['instructors_job_title'][i]) )
  return important_features


In [55]:
# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#show the data 
df.head(10)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,num_subscribers,avg_rating,num_reviews,created,important_features
0,1117836,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple,€94.99,1178,3.500000,128,2017-02-17T12:46:11Z,NoSQL Databases For Beginners <p>NoSQL (Non-SQ...
1,1117864,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist,€89.99,5,0.000000,0,2017-02-17T13:28:15Z,Tai Chi Foundation Course <p>Whether you're a ...
2,1117988,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De...",€39.99,1345,4.000000,7,2017-02-17T15:50:16Z,Music Pedagogy: 7 advices to teach a musical i...
3,1118012,Ultimate Power Electronics for Electrical Powe...,"<p><strong>""Ultimate Power Electronics Course ...",Learn everything about power electronics for e...,Teaching & Academics,Engineering,Ahmed Mahdy / Khadija Academy,Electrical Power Engineer,€94.99,3186,4.592593,481,2017-02-17T16:15:02Z,Ultimate Power Electronics for Electrical Powe...
4,1118046,Facebook Advertising: Create Converting Facebo...,<p><strong>Over 8000 students have joined this...,"Learn how to select your niche, identify marke...",Marketing,Social Media Marketing,Barry North,"Award Winning ""Sexiest Voice""",€89.99,10498,4.200000,68,2017-02-17T16:48:28Z,Facebook Advertising: Create Converting Facebo...
5,1118072,Planning & Scheduling: Be the Professional fro...,<p><strong>Planning and Scheduling</strong> is...,Project Planning and Scheduling Management cou...,Business,Project Management,"Construction Project Management Guru (PSP, PMI...",Promoting Professionalism in Planning and Sche...,€109.99,3393,4.074074,615,2017-02-17T17:08:03Z,Planning & Scheduling: Be the Professional fro...
6,1118146,Wie du einen genialen Vortrag hältst,<p>Dein Vortrag ist deine Chance</p> <p>Mit d...,Mit Simon (Profi-Speaker) und Michi (NLP-Train...,Personal Development,Career Development,Simon Schnetzer & Michael Schmidt,Speaker & Persönlichkeitsentwickler,€44.99,345,4.100000,31,2017-02-17T17:50:36Z,Wie du einen genialen Vortrag hältst <p>Dein V...
7,1118268,Top Five Leadership Behaviors,<p>Every Leader Needs To Gain Their Team's Tru...,Strong Leaders Don't Just Happen: Learn These ...,Personal Development,Leadership,Donna Galatas,HR Vice Empress / CEO of The Galatas Group,€94.99,129,4.350000,27,2017-02-17T19:38:01Z,Top Five Leadership Behaviors <p>Every Leader ...
8,1118274,The Visual Guide on How Neural Networks Learn ...,<p>Course Achievements (January 2021):&nbsp;</...,The BEST Resource for Understanding Neural Net...,Development,Data Science,Mauricio Maroto,"+10,000 students and growing!",€94.99,3163,4.700000,143,2017-02-17T19:54:28Z,The Visual Guide on How Neural Networks Learn ...
9,1118300,Aprende a crear Plugins para WordPress desde c...,<p><strong>Más de 60 reseñas fantásticas !!! 4...,Es hora de saltar al siguiente paso en WordPre...,Development,Web Development,Gilbert Rodríguez,Desarrollador Web,€94.99,1457,4.500000,243,2017-02-17T20:32:46Z,Aprende a crear Plugins para WordPress desde c...


In [56]:
# creating stop word
stop_words =  stopwords.words('arabic') + stopwords.words('azerbaijani') + stopwords.words('danish') + stopwords.words('dutch') + stopwords.words('english') + stopwords.words('finnish') + stopwords.words('french') + stopwords.words('german') + stopwords.words('greek') + stopwords.words('hungarian') + stopwords.words('indonesian') + stopwords.words('italian') + stopwords.words('kazakh') + stopwords.words('nepali') + stopwords.words('norwegian') + stopwords.words('portuguese') + stopwords.words('romanian') + stopwords.words('russian') + stopwords.words('slovene') + stopwords.words('spanish') + stopwords.words('swedish') + stopwords.words('tajik') + stopwords.words('turkish')
print(stop_words)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [57]:
# convert the text to a matrix of token counts 
df['important_features'].replace('<[^<]+?>',' ',regex=True, inplace = True)
print (df['important_features'].head(3))
df['important_features'] = df['important_features'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cm = CountVectorizer().fit_transform(df['important_features'])

# GEt the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)


0    NoSQL Databases For Beginners  NoSQL (Non-SQL ...
1    Tai Chi Foundation Course  Whether you're a be...
2    Music Pedagogy: 7 advices to teach a musical i...
Name: important_features, dtype: object
[[1.         0.08476291 0.14103213 ... 0.00295429 0.08553797 0.14755645]
 [0.08476291 1.         0.17646815 ... 0.0081325  0.10360544 0.06181151]
 [0.14103213 0.17646815 1.         ... 0.         0.14061335 0.1262184 ]
 ...
 [0.00295429 0.0081325  0.         ... 1.         0.         0.06139903]
 [0.08553797 0.10360544 0.14061335 ... 0.         1.         0.08607986]
 [0.14755645 0.06181151 0.1262184  ... 0.06139903 0.08607986 1.        ]]


In [58]:
# save data in file 
data = asarray(cs)
save('data.npy', data)


In [59]:
# Loading data from a file
cs = load('data.npy')

In [60]:
# Get the shape of the cosine similarity matrix 
cs.shape


(3945, 3945)

In [69]:
# Get the title of the course that the user likes 
title_to_search = 'Tai Chi Foundation Course'

# Find the course id
course_id = df[df.title == title_to_search]['Course_id'].index
print (course_id[0])
#course_id = 251
# IndexError: index 0 is out of bounds for axis 0 with size 0
# this means that the title doesn't existe

1


In [70]:
# Create a list of enumeration for the similarity score [(course_id, similiraty score ),(..)]
scores = list(enumerate(cs[course_id[0]]))

In [71]:
#sort the list 
sorted_scores = sorted(scores, key = lambda x:x[1], reverse  = True)
sorted_scores = sorted_scores[1:]


In [72]:
#Print the sorted scores
print(sorted_scores)

[(1836, 0.47288048219948653), (2585, 0.33518006953329693), (1329, 0.28425233161454216), (1894, 0.25244825695711365), (2403, 0.25030431391608343), (1274, 0.24616183130937527), (3568, 0.24415567358823811), (3853, 0.23952689687873938), (1717, 0.23630585231952217), (2208, 0.2348697567939048), (2815, 0.23268787336008676), (1109, 0.22895136135226085), (1479, 0.22742361270862907), (631, 0.2268157757864524), (1302, 0.2266446142201822), (1593, 0.2209709274744873), (568, 0.22088709735120957), (3482, 0.2184584869370109), (3119, 0.2161507621432144), (417, 0.21569819894287398), (2868, 0.21549920950447166), (915, 0.21349662311335998), (3326, 0.21173185698978206), (2624, 0.21098736814243052), (729, 0.20994645518930363), (1904, 0.20890109550954555), (18, 0.2086602373481967), (3188, 0.20584674239815465), (2913, 0.20499596655422742), (2812, 0.20414906729358745), (2864, 0.2030164812207561), (2066, 0.20044294991680828), (1299, 0.2001947038497876), (2768, 0.20014175823664762), (494, 0.19939055216355978), (

In [81]:
# Create a loop to print the first 7 similar movies
j = 0
sorted_by_nomber_of_subscriber= []
print('The 7 most recommended courses : ', title_to_search, '\nare:\n')
print('ID  Course Title' )
for item in sorted_scores:
  course_title = df[df.index == item[0]]['title'].values[0]
  course_id = df[df.index == item[0]]['Course_id'].values[0]
  #course_headline = df[df.index == item[0]]['headline'].values[0]
  num_subscribers = df[df.index == item[0]]['num_subscribers'].values[0]
  created = df[df.index == item[0]]['created'].values[0]
  print(course_id, course_title, " | " + str(num_subscribers) )
  j=j+1
  d = {"course_id": course_id, "course_title": course_title, "num_subscribers": num_subscribers, "created": created }
  sorted_by_nomber_of_subscriber.append(d)
  if j>10:
    break

The 7 most recommended courses :  Tai Chi Foundation Course 
are:

ID  Course Title
1184426 Tai Chi 5 Minutes a Day Beginners Course  | 94
1208274 Tai Chi for Absolute Beginners! (Part 1)  | 657
1167858 Healing Sounds Qi Gong Meditation  | 819
1186194 Launch Your First UDEMY Course In 7 Days or Less -Unofficial  | 9105
1202758 Learn The Basics of Kenpo Karate 1  | 196
1166482 Learn Meditation - Theory & Practice  | 15919
1240834 Introduction to Qigong for health and longevity  | 28
1249018 Java for beginners (Core concepts): Easy course on Java  | 12589
1180700 Optimal Health - Your Journey for a Long & Happy Life  | 6040
1196768 Fitness and Health : Improve Fasciae Fitness with Qi Gong  | 209
1215322 Indonesian for Beginners: Level 2 (A2)  | 316


In [82]:
print('soreted by number of subscriber : ', title_to_search, '\nare:\n')
print(sorted_by_nomber_of_subscriber)
sorted_by_nomber_of_subscriber_updated = sorted(sorted_by_nomber_of_subscriber, key=lambda k: k['num_subscribers'], reverse=True)
print(sorted_by_nomber_of_subscriber_updated)

soreted by number of subscriber :  Tai Chi Foundation Course 
are:

[{'course_id': 1184426, 'course_title': 'Tai Chi 5 Minutes a Day Beginners Course', 'num_subscribers': 94, 'created': '2017-04-18T04:27:58Z'}, {'course_id': 1208274, 'course_title': 'Tai Chi for Absolute Beginners! (Part 1)', 'num_subscribers': 657, 'created': '2017-05-08T07:04:40Z'}, {'course_id': 1167858, 'course_title': 'Healing Sounds Qi Gong Meditation', 'num_subscribers': 819, 'created': '2017-04-03T14:23:46Z'}, {'course_id': 1186194, 'course_title': 'Launch Your First UDEMY Course In 7 Days or Less -Unofficial', 'num_subscribers': 9105, 'created': '2017-04-19T15:34:55Z'}, {'course_id': 1202758, 'course_title': 'Learn The Basics of Kenpo Karate 1', 'num_subscribers': 196, 'created': '2017-05-03T14:09:37Z'}, {'course_id': 1166482, 'course_title': 'Learn Meditation - Theory & Practice', 'num_subscribers': 15919, 'created': '2017-04-02T06:10:14Z'}, {'course_id': 1240834, 'course_title': 'Introduction to Qigong for h

In [83]:
for item in sorted_by_nomber_of_subscriber_updated:
  print(item['course_id'], item['course_title'], " | " + str(item['num_subscribers']))

1166482 Learn Meditation - Theory & Practice  | 15919
1249018 Java for beginners (Core concepts): Easy course on Java  | 12589
1186194 Launch Your First UDEMY Course In 7 Days or Less -Unofficial  | 9105
1180700 Optimal Health - Your Journey for a Long & Happy Life  | 6040
1167858 Healing Sounds Qi Gong Meditation  | 819
1208274 Tai Chi for Absolute Beginners! (Part 1)  | 657
1215322 Indonesian for Beginners: Level 2 (A2)  | 316
1196768 Fitness and Health : Improve Fasciae Fitness with Qi Gong  | 209
1202758 Learn The Basics of Kenpo Karate 1  | 196
1184426 Tai Chi 5 Minutes a Day Beginners Course  | 94
1240834 Introduction to Qigong for health and longevity  | 28


In [87]:
sorted_by_created_dated = sorted(sorted_by_nomber_of_subscriber_updated, key=lambda k: k['created'], reverse=True)
for item in sorted_by_created_dated:
  print(item['course_id'], item['course_title'], " | " + str(item['created']), " | " + str(item['num_subscribers']))

1249018 Java for beginners (Core concepts): Easy course on Java  | 2017-06-10T20:03:40Z  | 12589
1240834 Introduction to Qigong for health and longevity  | 2017-06-04T04:22:56Z  | 28
1215322 Indonesian for Beginners: Level 2 (A2)  | 2017-05-14T08:53:47Z  | 316
1208274 Tai Chi for Absolute Beginners! (Part 1)  | 2017-05-08T07:04:40Z  | 657
1202758 Learn The Basics of Kenpo Karate 1  | 2017-05-03T14:09:37Z  | 196
1196768 Fitness and Health : Improve Fasciae Fitness with Qi Gong  | 2017-04-28T10:09:02Z  | 209
1186194 Launch Your First UDEMY Course In 7 Days or Less -Unofficial  | 2017-04-19T15:34:55Z  | 9105
1184426 Tai Chi 5 Minutes a Day Beginners Course  | 2017-04-18T04:27:58Z  | 94
1180700 Optimal Health - Your Journey for a Long & Happy Life  | 2017-04-14T13:34:01Z  | 6040
1167858 Healing Sounds Qi Gong Meditation  | 2017-04-03T14:23:46Z  | 819
1166482 Learn Meditation - Theory & Practice  | 2017-04-02T06:10:14Z  | 15919
